In [1]:
import numpy as np
from stable_baselines3 import A2C
from gymnasium.spaces import Box
from poke_env.data import GenData
import asyncio

from poke_env.player import Gen9EnvSinglePlayer, RandomPlayer, SimpleHeuristicsPlayer
from poke_env import AccountConfiguration, ShowdownServerConfiguration


C:\Users\joewa\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Nick here is your code snip-it

In [2]:
MAX_TURNS=255
class SimpleRLPlayer(Gen9EnvSinglePlayer):
    def embed_battle(self, battle):
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=GEN_9_DATA.type_chart
                )

        # We count how many pokemons have not fainted in each team
        remaining_mon_team = (
            len([mon for mon in battle.team.values() if mon.fainted]) / 6
        )
        remaining_mon_opponent = (
            len([mon for mon in battle.opponent_team.values() if mon.fainted]) / 6
        )

        # hp
        remaining_health = np.ones(6)
        opponent_remaining_health = np.ones(6)
        # stats
        stats = np.zeros(7)
        opponent_stats = np.zeros(7)
        # status
        status = np.zeros(6)
        opponent_status = [mon.status for mon in battle.opponent_team.values()]
        for i, mon in enumerate(battle.team.values()):
            remaining_health[i] = mon.current_hp_fraction
            if mon.status is not None:
                status[i] = mon.status.value
            if mon.active:
                for j,b in enumerate(mon.boosts.values()):
                    stats[j] = b
        for i, mon in enumerate(battle.opponent_team.values()):
            opponent_remaining_health[i] = mon.current_hp_fraction
            if mon.status is not None:
                opponent_status[i] = mon.status.value
            if mon.active:
                for j,b in enumerate(mon.boosts.values()):
                    opponent_stats[j] = b

        # tera
        tera = bool(battle.can_tera)
        opponent_tera = bool(battle._opponent_can_terrastallize)

        # hazards
        hazarddex = { # relevant side conditions to track (to reduce this dimension by half and not have variable sized input)
            SideCondition.UNKNOWN:0,
            SideCondition.AURORA_VEIL:1,
            SideCondition.LIGHT_SCREEN:2,
            SideCondition.MIST:3,
            SideCondition.REFLECT:4,
            SideCondition.SAFEGUARD:5,
            SideCondition.SPIKES:6,
            SideCondition.STEALTH_ROCK:7,
            SideCondition.STICKY_WEB:8,
            SideCondition.TAILWIND:9,
            SideCondition.TOXIC_SPIKES:10,
        }
        hazards = np.zeros(11)
        for cond,val in battle.side_conditions.items():
            if cond in hazarddex:
                hazards[hazarddex[cond]] = val
        opponent_hazards = np.zeros(11)
        for cond,val in battle.opponent_side_conditions.items():
            if cond in hazarddex:
                opponent_hazards[hazarddex[cond]] = val

        # terrain
        fields = np.zeros(13)
        for f,val in battle.fields.items():
            fields[f.value-1] = val

        # weather
        weather = np.zeros(9)
        for w,val in battle.weather.items():
            weather[w.value-1] = val

        # turn
        turn = battle.turn

        # Final vector with 10 components
        return np.concatenate(
            [
                moves_base_power, # 4* -1,3
                moves_dmg_multiplier, # 4* 0,4
                [remaining_mon_team, remaining_mon_opponent], # 1*1* 0,1 0,1
                remaining_health, # 6* 0,1
                opponent_remaining_health, # 6* 0,1
                [tera, opponent_tera], # 1*1* 0,1 0,1
                stats, # 7* -6,6
                opponent_stats, # 7* -6,6
                hazards, # 11* 0,MAX_TURNS
                opponent_hazards, # 11* 0,MAX_TURNS
                fields, # 13* 0,MAX_TURNS
                weather, # 9* 0,MAX_TURNS
                [turn], # 1* 0,MAX_TURNS
            ]
        )

    def calc_reward(self, last_state, current_state) -> float:
        return self.reward_computing_helper(
            current_state, fainted_value=2, hp_value=1, victory_value=30
        )+25/current_state.turn
    
    def describe_embedding(self):
        low = np.concatenate([[-1]*4, [0]*4, [0]*2, [0]*6, [0]*6, [0]*2, [-6]*7, [-6]*7, [0]*11, [0]*11, [0]*13, [0]*9, [0]])
        high = np.concatenate([[3]*4, [4]*4, [1]*2, [1]*6, [1]*6, [1]*2, [6]*7, [6]*7, [MAX_TURNS]*11, [MAX_TURNS]*11, [MAX_TURNS]*13, [MAX_TURNS]*9, [MAX_TURNS]])
        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32,
        )


class MaxDamagePlayer(RandomPlayer):
    def choose_move(self, battle):
        # If the player can attack, it will
        if battle.available_moves:
            # Finds the best move among available ones
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            return self.create_order(best_move)

        # If no attack is available, a random switch will be made
        else:
            return self.choose_random_move(battle)


In [ ]:
# This is the function that will be used to train the a2c

NB_TRAINING_STEPS = 100
TEST_EPISODES = 100
GEN_9_DATA = GenData.from_gen(9)

opponent = RandomPlayer()
third_opponent = SimpleHeuristicsPlayer()
second_opponent = MaxDamagePlayer()

env_player = SimpleRLPlayer(opponent=opponent)

model = A2C("MlpPolicy", env_player, verbose=1)
model.learn(total_timesteps=NB_TRAINING_STEPS)
model.env_opponent=third_opponent
model.learn(NB_TRAINING_STEPS)
model.env._opponent=second_opponent
model.learn(NB_TRAINING_STEPS)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


2024-11-29 10:48:51,934 - SimpleRLPlayer 1 - ERROR - Unhandled exception raised while handling message:
>battle-gen9randombattle-73173
|player|p2|RandomPlayer 1|170|
|teamsize|p1|6
|teamsize|p2|6
|gen|9
|tier|[Gen 9] Random Battle
|rule|Species Clause: Limit one of each Pokémon
|rule|HP Percentage Mod: HP is shown in percentages
|rule|Sleep Clause Mod: Limit one foe put to sleep
|rule|Illusion Level Mod: Illusion disguises the Pokémon's true level
|
|t:|1732898931
|start
|switch|p1a: Smeargle|Smeargle, L95, F|258/258
|switch|p2a: Dachsbun|Dachsbun, L92, M|100/100
|turn|1
Traceback (most recent call last):
  File "c:\Users\joewa\anaconda3\envs\Project\lib\site-packages\poke_env\ps_client\ps_client.py", line 135, in _handle_message
    await self._handle_battle_message(split_messages)  # type: ignore
  File "c:\Users\joewa\anaconda3\envs\Project\lib\site-packages\poke_env\player\player.py", line 351, in _handle_battle_message
    await self._handle_battle_request(battle)
  File "c:\Users

In [ ]:
obs, reward, done, _, info = env_player.step(0)
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env_player.step(action)

finished_episodes = 0

env_player.reset_battles()
obs, _ = env_player.reset()
while True:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env_player.step(action)

    if done:
        finished_episodes += 1
        if finished_episodes >= TEST_EPISODES:
            break
        obs, _ = env_player.reset()

print("Won", env_player.n_won_battles, "battles against", env_player._opponent)

finished_episodes = 0

env_player.reset_env(restart=True, opponent=MaxDamagePlayer())
while True:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env_player.step(action)

    if done:
        finished_episodes += 1
        obs, _ = env_player.reset()
        if finished_episodes >= TEST_EPISODES:
            break

print("Won", env_player.n_won_battles, "battles against", env_player._opponent)


In [ ]:
async def main():
    # We create a random player
    player = SimpleRLPlayer(
        player_configuration=AccountConfiguration("Agent_ego", "RLclass")
        server_configuration=ShowdownServerConfiguration,
    )
    # Sending challenges to 'your_username'
    #await player.send_challenges("your_username", n_challenges=1)

    # Accepting one challenge from any user
    #await player.accept_challenges(None, 1)

    # Accepting three challenges from 'your_username'
    #await player.accept_challenges('your_username', 3)

    # Playing 5 games on the ladder
    await player.ladder

    # Print the rating of the player and its opponent after each battle
    for battle in player.battles.values():
        print(battle.rating, battle.opponent_rating)

asyncio.get_event_loop().run_until_complete(main())

In [ ]:
# n_won_battles and n_finished_battles

print(
    f"Player {random_player.username} won {random_player.n_won_battles} out of {random_player.n_finished_battles} played"
)
print(
    f"Player {second_player.username} won {second_player.n_won_battles} out of {second_player.n_finished_battles} played"
)

# Looping over battles

for battle_tag, battle in random_player.battles.items():
    print(battle_tag, battle.won)

Player RandomPlayer 1 won 0 out of 1 played
Player RandomPlayer 2 won 1 out of 1 played
battle-gen9randombattle-438 False


You can look at more properties of the [Player](../modules/player.rst) and [Battle](../modules/battle.rst) classes in the documentation.

### Running a Cross-Evaluation

`poke-env` provides a `cross_evaluate` function, that allows you to run a cross evaluation between multiple agents. It will run a number of battles between the two agents, and return the results of the evaluation in a structured way.

In [ ]:
from poke_env import cross_evaluate

third_player = RandomPlayer()

players = [random_player, second_player, third_player]

cross_evaluation = await cross_evaluate(players, n_challenges=5)
cross_evaluation

{'RandomPlayer 1': {'RandomPlayer 1': None,
  'RandomPlayer 2': 0.5,
  'RandomPlayer 3': 0.6},
 'RandomPlayer 2': {'RandomPlayer 1': 0.5,
  'RandomPlayer 2': None,
  'RandomPlayer 3': 0.6},
 'RandomPlayer 3': {'RandomPlayer 1': 0.4,
  'RandomPlayer 2': 0.4,
  'RandomPlayer 3': None}}

Here's one way to pretty print the results of the cross evaluation using `tabulate`:

In [ ]:
from tabulate import tabulate

table = [["-"] + [p.username for p in players]]
for p_1, results in cross_evaluation.items():
    table.append([p_1] + [cross_evaluation[p_1][p_2] for p_2 in results])

print(tabulate(table))

--------------  --------------  --------------  --------------
-               RandomPlayer 1  RandomPlayer 2  RandomPlayer 3
RandomPlayer 1                  0.5             0.6
RandomPlayer 2  0.5                             0.6
RandomPlayer 3  0.4             0.4
--------------  --------------  --------------  --------------


## Building a Max Damage Player

In this section, we introduce the `MaxDamagePlayer`, a custom agent designed to choose moves that maximize damage output.

### Implementing the MaxDamagePlayer Class

The primary task is to override the choose_move method. This method, defined as `choose_move(self, battle: Battle) -> str`, requires a `Battle` object as input, representing the current game state, and outputs a move order as a string. This move order must adhere to the [showdown protocol](https://github.com/smogon/pokemon-showdown/blob/master/sim/SIM-PROTOCOL.md) format. The `poke-env` library provides the `create_order` method to assist in formatting move orders directly from `Pokemon` and `Move` objects.

The `battle` parameter, a `Battle` object, encapsulates the agent's current knowledge of the game state. It provides various properties for easy access to game details, such as `active_pokemon`, `available_moves`, `available_switches`, `opponent_active_pokemon`, `opponent_team`, and `team`.

For this example, we'll utilize `available_moves`, which gives us a list of `Move` objects available in the current turn.

Our focus in implementing `MaxDamagePlayer` involves two key steps: interpreting the game state information from the battle object and then generating and returning a correctly formatted move order.

In [ ]:
from poke_env.player import Player


class MaxDamagePlayer(Player):
    def choose_move(self, battle):
        # Chooses a move with the highest base power when possible
        if battle.available_moves:
            # Iterating over available moves to find the one with the highest base power
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            # Creating an order for the selected move
            return self.create_order(best_move)
        else:
            # If no attacking move is available, perform a random switch
            # This involves choosing a random move, which could be a switch or another available action
            return self.choose_random_move(battle)

In the `choose_move` method, our first step is to determine if there are any available moves for the current turn, as indicated by `battle.available_moves`. When a move is available, we select the one with the highest `base_power`. Formatting our choice is achieved by the `create_order`.

However, there are scenarios where no moves are available. In such cases, we use `choose_random_move(battle)`. This method randomly selects either a move or a switch, and guarantees that we will return a valid order.

The `Player.create_order` function is a crucial part of this process. It's a wrapper method that generates valid battle messages. It can take either a `Move` or a `Pokemon` object as its input. When passing a `Move` object, additional parameters such as `mega`, `z_move`, `dynamax`, or `terastallize` can be specified to indicate special battle actions.

We will adjust our strategy to include `terastallize` at the earliest opportunity, enhancing the effectiveness of our player in battle scenarios.

In [ ]:
class MaxDamagePlayer(Player):
    def choose_move(self, battle):
        if battle.available_moves:
            best_move = max(battle.available_moves, key=lambda move: move.base_power)

            if battle.can_tera:
                return self.create_order(best_move, terastallize=True)

            return self.create_order(best_move)
        else:
            return self.choose_random_move(battle)

### Testing the MaxDamagePlayer

Next, we'll test our `MaxDamagePlayer` against a `RandomPlayer` in a series of battles:


In [ ]:
# Creating players
random_player = RandomPlayer()
max_damage_player = MaxDamagePlayer()

# Running battles
await max_damage_player.battle_against(random_player, n_battles=100)

# Displaying results
print(f"Max damage player won {max_damage_player.n_won_battles} / 100 battles")

CancelledError: 

Unsurprisingly, the `MaxDamagePlayer` wins most of the battles.

## Setting teams

Most formats do not provide a team automatically. 

To specify a team, you have two main options: you can either provide a `str` describing your team, or a `Teambuilder` object. This example will focus on the first option; if you want to learn more about using teambuilders, please refer to [Creating a custom teambuilder and Teambuilder: Parse, manage and generate showdown teams](using_a_custom_teambuilder.ipynb).

The easiest way to specify a team in poke-env is to copy-paste a showdown team. You can use showdown’s teambuilder and export it directly.

Alternatively, you can use showdown’s packed formats, which correspond to the actual string sent by the showdown client to the server.

### Using a `str`

Here's an example:

In [ ]:
team_1 = """
Goodra (M) @ Assault Vest
Ability: Sap Sipper
EVs: 248 HP / 252 SpA / 8 Spe
Modest Nature
IVs: 0 Atk
- Dragon Pulse
- Flamethrower
- Sludge Wave
- Thunderbolt

Sylveon (M) @ Leftovers
Ability: Pixilate
EVs: 248 HP / 244 Def / 16 SpD
Calm Nature
IVs: 0 Atk
- Hyper Voice
- Mystical Fire
- Protect
- Wish

Toxtricity (M) @ Throat Spray
Ability: Punk Rock
EVs: 4 Atk / 252 SpA / 252 Spe
Rash Nature
- Overdrive
- Boomburst
- Shift Gear
- Fire Punch

Seismitoad (M) @ Leftovers
Ability: Water Absorb
EVs: 252 HP / 252 Def / 4 SpD
Relaxed Nature
- Stealth Rock
- Scald
- Earthquake
- Toxic

Corviknight (M) @ Leftovers
Ability: Pressure
EVs: 248 HP / 80 SpD / 180 Spe
Impish Nature
- Defog
- Brave Bird
- Roost
- U-turn

Galvantula @ Focus Sash
Ability: Compound Eyes
EVs: 252 SpA / 4 SpD / 252 Spe
Timid Nature
IVs: 0 Atk
- Sticky Web
- Thunder Wave
- Thunder
- Energy Ball
"""
team_2 = """
Togekiss @ Leftovers
Ability: Serene Grace
EVs: 248 HP / 8 SpA / 252 Spe
Timid Nature
IVs: 0 Atk
- Air Slash
- Nasty Plot
- Substitute
- Thunder Wave

Galvantula @ Focus Sash
Ability: Compound Eyes
EVs: 252 SpA / 4 SpD / 252 Spe
Timid Nature
IVs: 0 Atk
- Sticky Web
- Thunder Wave
- Thunder
- Energy Ball

Cloyster @ Leftovers
Ability: Skill Link
EVs: 252 Atk / 4 SpD / 252 Spe
Adamant Nature
- Icicle Spear
- Rock Blast
- Ice Shard
- Shell Smash

Sandaconda @ Focus Sash
Ability: Sand Spit
EVs: 252 Atk / 4 SpD / 252 Spe
Jolly Nature
- Stealth Rock
- Glare
- Earthquake
- Rock Tomb

Excadrill @ Focus Sash
Ability: Sand Rush
EVs: 252 Atk / 4 SpD / 252 Spe
Adamant Nature
- Iron Head
- Rock Slide
- Earthquake
- Rapid Spin

Cinccino @ Leftovers
Ability: Skill Link
EVs: 252 Atk / 4 Def / 252 Spe
Jolly Nature
- Bullet Seed
- Knock Off
- Rock Blast
- Tail Slap
"""

p1 = MaxDamagePlayer(battle_format="gen8ou", team=team_1)
p2 = MaxDamagePlayer(battle_format="gen8ou", team=team_2)

await p1.battle_against(p2, n_battles=1)

### Dealing with team preview

By default, teampreview will be handled by randomly selecting the order of your pokemons. You can change this behaviour by overriding the `teampreview` method of the `Player` class. Here is an example using type-based heuristics:

In [ ]:
import numpy as np


def teampreview_performance(mon_a, mon_b):
    # We evaluate the performance on mon_a against mon_b as its type advantage
    a_on_b = b_on_a = -np.inf
    for type_ in mon_a.types:
        if type_:
            a_on_b = max(
                a_on_b,
                type_.damage_multiplier(
                    *mon_b.types, type_chart=GenData.from_gen(8).type_chart
                ),
            )
    # We do the same for mon_b over mon_a
    for type_ in mon_b.types:
        if type_:
            b_on_a = max(
                b_on_a,
                type_.damage_multiplier(
                    *mon_a.types, type_chart=GenData.from_gen(8).type_chart
                ),
            )
    # Our performance metric is the different between the two
    return a_on_b - b_on_a


class MaxDamagePlayerWithTeampreview(MaxDamagePlayer):
    def teampreview(self, battle):
        mon_performance = {}

        # For each of our pokemons
        for i, mon in enumerate(battle.team.values()):
            # We store their average performance against the opponent team
            mon_performance[i] = np.mean(
                [
                    teampreview_performance(mon, opp)
                    for opp in battle.opponent_team.values()
                ]
            )

        # We sort our mons by performance
        ordered_mons = sorted(mon_performance, key=lambda k: -mon_performance[k])

        # We start with the one we consider best overall
        # We use i + 1 as python indexes start from 0
        #  but showdown's indexes start from 1
        return "/team " + "".join([str(i + 1) for i in ordered_mons])


p3 = MaxDamagePlayerWithTeampreview(battle_format="gen8ou", team=team_1)
p4 = MaxDamagePlayerWithTeampreview(battle_format="gen8ou", team=team_2)

await p3.battle_against(p4, n_battles=1)

## Other Initialization Options for `Player` Objects

### Specifying an Avatar

You can specify an `avatar` argument when initializing a `Player` object. This argument is a string, corresponding to the avatar's name.

You can find a [list of avatar names here](https://github.com/smogon/pokemon-showdown-client/blob/6d55434cb85e7bbe614caadada819238190214f6/play.pokemonshowdown.com/src/battle-dex-data.ts#L690). If the avatar you are looking for is not in this list, you can inspect the message the client is sending to the server by opening your browser's development console and selecting the avatar manually.


In [ ]:
player_with_avatar = RandomPlayer(avatar="boarder")

### Saving Battle Replays

You can save battle replays by specifying a `save_replay` value when initializing a `Player` object. This argument can either be a boolean (if `True`, the replays will be saved in the `replays`) or a string - in which case the replays will be saved in the specified directory.

In [ ]:
player_with_replays = RandomPlayer(save_replays="my_folder")

### Logging

Every `Player` instance has a custom logger. By default, it will only surface warnings and errors. You can change the logging level by specifying a `log_level` argument when initializing a `Player` object.

The two most relevant values are `logging.INFO` or 20, which will surface every message sent or received by the client (which is very useful when debugging) and 25, which is a custom level used by `poke-env` to surface only the most relevant events.

You can also use `logging.DEBUG` or 10, but the difference with `logging.INFO` should only be relevant for `poke-env` internals.

In [ ]:
verbose_player = RandomPlayer(log_level=20)

from asyncio import sleep

await sleep(1)

### Concurrency

By default, a `poke-env` `Player` will only run a single battle at a time. You can change this behavior by specifying a `max_concurrent_battles` argument when initializing a `Player` object.

This argument is an integer, and represents the maximum number of battles a `Player` can run at the same time. If 0, no limit will be enforced.

This can provide a significant speedup when your process is not CPU bound.

In [ ]:
import time

# Time to run 50 battles, one at a time
start = time.time()
await random_player.battle_against(second_player, n_battles=50)
end = time.time()
print(f"Time to run 50 battles, one at a time: {end - start:.2f} seconds")

In [ ]:
unrestricted_random_player = RandomPlayer(max_concurrent_battles=0)
unrestricted_second_player = RandomPlayer(max_concurrent_battles=0)

# Time to run 50 battles, in parallel
start = time.time()
await unrestricted_random_player.battle_against(
    unrestricted_second_player, n_battles=50
)
end = time.time()
print(f"Time to run 50 battles, in parallel: {end - start:.2f} seconds")

Other options can also be used on the server side to make battles run faster.

### Pokemon Showdown Timer

You can turn on the Pokemon Showdown timer by setting `start_timer_on_battle_start` to `True` when initializing a `Player` object.

This is mostly relevant when pitting your argents against humans.

In [ ]:
impatient_player = RandomPlayer(start_timer_on_battle_start=True)